In [ ]:
!pip install gitpython

In [ ]:
import os

# 현재 작업 디렉토리 확인
current_directory = os.getcwd()

# 현재 디렉토리의 파일 목록 출력
file_list = os.listdir(current_directory)
print(file_list)


In [ ]:
from git import Repo

# GitHub 저장소의 URL
REPO_URL = 'https://github.com/USERNAME/REPO_NAME.git'
# 파일을 업로드할 경로
FILE_PATH = 'path/to/your/file.ipynb'
# 업로드할 파일의 커밋 메시지
COMMIT_MESSAGE = 'Upload file from Colab'

# GitHub 저장소를 클론합니다
Repo.clone_from(REPO_URL, '/content/repo')

# 업로드할 파일을 복사합니다
!cp FILE_PATH /content/repo/

# 변경된 파일을 커밋하고 푸시합니다
repo = Repo('/content/repo')
repo.git.add('.')
repo.git.commit('-m', COMMIT_MESSAGE)
repo.git.push()

목표: 고객들의 특성 및 신용 정보를 바탕으로,정기예금(Term Deposit) 가입 여부 예측하고 정기예금 유도에 영향을 미칠 주요한 변수들을 파악 후
활용하여 주어진 데이터셋으로 서로다른 머신러닝 알고리즘 최소 2개를 비교한다.





다음과 같은 순서로 분석을 진행하고자 한다.

<데이터파악>
1. 데이터 생김새와 변수 파악: 포르투갈 은행의 텔레마케팅을 통한 정기 예금 가입 유도 전략 (정기 예금 가입 예측 모형)
2. 변수확인: 데이터의 주요 특징이되는 변수들을 파악
3.시각화 및 기술통계량 요약등을 통해 개략적으로 데이터 생김새를 파악

<모델별 실습>
4. 데이터 전처리
5. 데이터 세트 분리
7. 데이터세트 분리 : 학습데이터와 테스트 데이터로 분리
8. 모델학습: 학습데이터를 기반으로 머신러닝 알고리즘을 적용하여 모델을 학습
9. 예측수행: 학습된 모델을 이용하여 테스트 데이터의 결과 확인
10. 평가: 예측된 결과값과 테스트 데이터의 실제 결과값을 비교해 모델 성능 평가

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Database Phase
import pandas as pd
import numpy as np

# Machine Learning Phase
import sklearn
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

#Metrics Phase
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score

#Visualization Phase
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.pylab as pylab
%matplotlib inline
pd.set_option('display.max_columns', 500)
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# **데이터 생김새와 변수 파악: 포르투갈 은행의 텔레마케팅을 통한 정기 예금 가입 유도전략 (정기예금 가입 예측 모형)**




In [ ]:
import numpy as np  # numpy 패키지를 np로 불러옴
import pandas as pd  # pandas 패키지를 pd로 불러옴
from sklearn.metrics import accuracy_score  # scikit-learn에서 accuracy_score 함수를 불러옴
from xgboost import XGBClassifier  # XGBoost 모델을 불러옴
from sklearn.model_selection import train_test_split  # scikit-learn에서 train_test_split 함수를 불러옴
from sklearn.model_selection import StratifiedKFold  # scikit-learn에서 StratifiedKFold 함수를 불러옴
from sklearn.model_selection import cross_val_score  # scikit-learn에서 cross_val_score 함수를 불러옴
import matplotlib.pyplot as plt  # matplotlib 패키지를 plt로 불러옴
#별도의 창을열지않고 플롯 확인
%matplotlib inline

Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')  # bank.csv 파일을 pandas의 read_csv 함수를 이용해 불러옴
Original

In [ ]:
import pandas as pd
import seaborn as sns

# 상관관계 행렬 생성
correlation_matrix = Original.select_dtypes(include=['int64', 'float64']).corr()

# 상관관계 히트맵 생성
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('상관관계 행렬')
plt.show()

# pandas를 사용하여 데이터를 DataFrame으로 읽어옵니다.
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# DataFrame의 처음 몇 개의 행을 출력하여 데이터를 확인합니다.
print("데이터의 처음 몇 개 행:")
print(Original.head())

# DataFrame의 차원(행의 개수, 열의 개수)을 출력합니다.
print("\n데이터 형태:")
print(Original.shape)

# 수치형 열에 대한 요약 통계량(개수, 평균, 표준편차 등)을 계산합니다.
print("\n요약 통계량:")
print(Original.describe())

# DataFrame에서 결측값을 확인하고 각 열의 결측값 개수를 출력합니다.
print("\n결측값 확인:")
print(Original.isnull().sum())

# 각 열의 데이터 타입을 출력합니다.
print("\n데이터 타입:")
print(Original.dtypes)

# 범주형 열에 대해 고유값과 해당 개수를 출력합니다.
print("\n고유값 확인:")
for column in Original.select_dtypes(include='object').columns:
    print(f"\n{column}:")
    print(Original[column].value_counts())

# 수치형 열에 대한 히스토그램을 생성하여 데이터의 분포를 시각화합니다.
print("\n히스토그램:")
Original.hist(figsize=(10, 8))

# 수치형 열을 대상으로 상관관계 행렬 히트맵을 생성하여 변수 간 관계를 파악합니다.
print("\n상관관계 행렬 히트맵:")
correlation_matrix = Original.select_dtypes(include=['int64', 'float64']).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('상관관계 행렬')
plt.show()


In [ ]:
#Original의 정보를 파악, 데이터의 개수는 4,521개
Original.info()

**변수확인: 데이터의 주요 특징이되는 변수들을 파악**

마케팅 캠페인 방법
대상 상품: 정기예금 상품
고객의 선정: 정기예금 가입 여부(y)를 타겟으로 하는 분류모형을 이용,
가입가능성이 높은 고객을 선별
접촉채널: 선정된 고객에 유무선전화를 이용하여 상품 소개 후 정기예금 가입을 권유
데이터 수: 4,521

변수의 개수 : 17 (=16 + output)

|연번||이름||변수종류|변수명|변수형태|변수설명|
|------||---||------|---|---|---|
|0||age||설명변수|나이|이산형|19세~87세|
|1||job||설명변수|직장|명목형|노동자,기술자,서비스직 등 12가지|
|2||marital||설명변수|결혼|명목형|결혼상태(이혼,결혼,싱글,아직모름)|
|3||education||설명변수|교육|명목형|최종학력(고등학교,대학교 등 12가지)|
|4||default||설명변수|채무불이행|명목형|채무불이행여부 (맞다,아니다,모름)|
|5||balance||설명변수|잔고|연속형|고객 연평균 통장 잔고|
|6||housing||설명변수|주택대출|명목형|주택대출여부 (맞다,아니다,모름)|
|7||loan||설명변수|개인대출|명목형|개인대출여부 (맞다,아니다,모름)|
|8||contact||설명변수|전화형태|명목형|휴대폰,집전화|
|9||day||설명변수|요일|명목형|마지막으로 전화한 요일|
|10||month||설명변수|월|명목형|마지막으로 전화한 달|
|11||duration||설명변수|통화시간|연속형|고객과의 통화시간|
|12||campaign||설명변수|일수|이산형|과거 캠페인의 마지막 연락 후 이번 캠페 인에서 다시 연락 받을 때까지 일수|
|13||pdays||설명변수|캠페인|이산형|이번 캠페인동안 연락한 횟수|
|14||previous||설명변수|과거연락|연속형|캠페인 이전에 연락받았던 횟수|
|15||poutcome||설명변수|과거결과|명목형|과거 캠페인 때 정기 예금 가입 여부 (성공, 실패, 모름)|
|16||y||설명변수|결과|명목형|정기예금 가입결과(성공, 실패)|

**시각화 및 기술통계량 요약등을 통해 개략적으로 데이터 생김새를 파악**

'나이' 변수를 범주화로 시각화하여 어떤 연령대에 고객이 많이 분포되어 있는지 확인

In [ ]:
#pandas 라이브러리를 pd 라는 이름으로 가져온다.
import pandas as pd
#matplotlib 라이브러리를 plt라는 이름으로 가져온다.
import matplotlib.pyplot as plt
#matplotlib 라이브러리에서 font_manager가져오기
from matplotlib import font_manager

# 폰트 경로 지정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'

# 폰트 매니저에 폰트 등록
font_manager.fontManager.addfont(font_path)

# 적용할 폰트 설정
plt.rcParams['font.family'] = 'NanumBarunGothic'

# 데이터 읽기, 각 필드는 세미콜론으로 구분
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 히스토그램 그리기, bin수는 20으로 설정
plt.hist(Original['age'], bins=20)

# 축 레이블 설정
plt.xlabel('Age')
plt.ylabel('Frequency')

# 그래프 표시
plt.show()


**연령대별 정기예금 가입 분포를 확인**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 연령대 구분을 위한 함수
def age_group(age):
    if age < 30:
        return '20s'
    elif age < 40:
        return '30s'
    elif age < 50:
        return '40s'
    elif age < 60:
        return '50s'
    else:
        return '60s+'

# 'age' 열의 값을 기반으로 연령대를 생성하여 'age_group' 열을 추가
Original['age_group'] = Original['age'].apply(age_group)

# 연령대별 정기예금 가입 여부의 분포 계산
age_group_counts = Original.groupby(['age_group', 'y']).size().unstack()

# 폰트 경로 설정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 그래프 생성
age_group_counts.plot(kind='bar', stacked=True)
plt.xlabel('연령대', fontproperties=fontprop)
plt.ylabel('Count', fontproperties=fontprop)
plt.title('연령대별 정기예금 가입 여부 분포', fontproperties=fontprop)
plt.legend(['No', 'Yes'])
plt.show()


해석: 30대와 40대가 모수 자체가 많고 정기예금 가입자도 많은것으로 보인다

**'직업' 변수의 분포를 시각화로 표현**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# bank.csv 파일을 읽어와서 Original 변수에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 직업 데이터 추출
job_data = Original['job'].value_counts()

# 폰트 경로 설정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 그래프 생성
plt.bar(job_data.index, job_data.values)  # 직업별 막대 그래프 생성
plt.xticks(rotation=45, fontproperties=fontprop)  # x축 라벨 회전 및 폰트 지정
plt.xlabel('Job', fontproperties=fontprop)  # x축 라벨 지정 및 폰트 지정
plt.ylabel('Count', fontproperties=fontprop)  # y축 라벨 지정 및 폰트 지정
plt.title('Distribution of Jobs', fontproperties=fontprop)  # 그래프 제목 지정 및 폰트 지정
plt.show()  # 그래프 출력



**소득별로 정기예금 가입 분포를 확인**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# bank.csv 파일을 읽어와서 Original 변수에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 직업 데이터와 정기예금 가입 여부(y) 데이터 추출
job_deposit_data = Original.groupby(['job', 'y']).size().unstack()

# 폰트 경로 설정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 그래프 생성
job_deposit_data.plot(kind='bar', stacked=True)

plt.xlabel('Job', fontproperties=fontprop)  # x축 라벨 지정 및 폰트 지정
plt.ylabel('Count', fontproperties=fontprop)  # y축 라벨 지정 및 폰트 지정
plt.title('Distribution of Jobs by Deposit', fontproperties=fontprop)  # 그래프 제목 지정 및 폰트 지정
plt.legend(['No Deposit', 'Deposit'])  # 범례 지정
plt.xticks(rotation=45, fontproperties=fontprop)  # x축 라벨 회전 및 폰트 지정
plt.show()  # 그래프 출력



In [ ]:
#주부들이 많이 가입한것은 알겠는데 정확한 값이 그래프에 표현되면 좋겠다고 생각이 들었다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# bank.csv 파일을 읽어와서 Original 변수에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 직업 데이터와 정기예금 가입 여부(y) 데이터 추출
job_deposit_data = Original.groupby(['job', 'y']).size().unstack()

# 폰트 경로 설정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 그래프 생성
ax = job_deposit_data.plot(kind='bar', stacked=True)

plt.xlabel('Job', fontproperties=fontprop)  # x축 라벨 지정 및 폰트 지정
plt.ylabel('Count', fontproperties=fontprop)  # y축 라벨 지정 및 폰트 지정
plt.title('Distribution of Jobs by Deposit', fontproperties=fontprop)  # 그래프 제목 지정 및 폰트 지정
plt.legend(['No Deposit', 'Deposit'])  # 범례 지정
plt.xticks(rotation=45)  # x축 라벨 회전

# 막대 위에 수치 표시
for p in ax.patches:
    width = p.get_width()
    height = p.get_height()
    x, y = p.get_xy()
    ax.annotate(height, (x + width/2, y + height/2), ha='center', va='center')

plt.show()  # 그래프 출력



해석: 주부들이 가장 많이 가입했다.

고객들의 결혼여부 파악

In [ ]:
import pandas as pd

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 'marital' 변수만 따로 추출하여 marital_data에 저장
marital_data = Original['marital']

# 각 상태별 데이터 개수 카운트
married_count = (marital_data == 'married').sum()
divorced_count = (marital_data == 'divorced').sum()
single_count = (marital_data == 'single').sum()

# 각 상태별 데이터 비율 계산
married_percentage = round(married_count/len(marital_data)*100, 2)
divorced_percentage = round(divorced_count/len(marital_data)*100, 2)
single_percentage = round(single_count/len(marital_data)*100, 2)

# DataFrame 생성
data = {'Status': ['Married', 'Divorced', 'Single'],
        'Count': [married_count, divorced_count, single_count],
        'Percentage': [married_percentage, divorced_percentage, single_percentage]}
df = pd.DataFrame(data)

# DataFrame 출력
print(df)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 경로 설정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 그래프 생성
plt.bar(df['Status'], df['Count'])
plt.xticks(rotation=45)
plt.xlabel('Marital Status', fontproperties=fontprop)
plt.ylabel('Count', fontproperties=fontprop)
plt.title('Distribution of Marital Status', fontproperties=fontprop)
plt.show()


고객들의 결혼여부 데이터와 직업분포를 동시에 그래프로 표현

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 'job' 변수와 'marital' 변수 추출
job_data = Original['job']
marital_data = Original['marital']

# 각 직업별 결혼 상태 분포 계산
job_marital_data = pd.crosstab(job_data, marital_data, margins=True)

# 'All' 열 제거 (unknown 상태)
job_marital_data.drop('All', inplace=True)

# 폰트 경로 설정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 결혼 상태별 직업 분포 그래프 생성
job_marital_data.plot(kind='bar', stacked=True, figsize=(10,6))

# 그래프 제목, x축, y축 라벨링
plt.title('Marital Status by Job', fontproperties=fontprop)
plt.xlabel('Job', fontproperties=fontprop)
plt.ylabel('Count', fontproperties=fontprop)

# 그래프 출력
plt.show()



고객의 연평균 통장잔고를 시각화

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 'balance' 변수만 추출하여 balance_data에 저장
balance_data = Original['balance']

# 폰트 경로 설정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# scatter plot 그리기 (알파값은 투명도로 0은 투명, 1은 불투명을 뜻함)
plt.scatter(Original['age'], balance_data, alpha=0.2)

# 그래프 타이틀과 라벨링 추가
plt.title("Balance vs Age", fontproperties=fontprop)
plt.xlabel("Age", fontproperties=fontprop)
plt.ylabel("Balance", fontproperties=fontprop)

# 그래프 출력
plt.show()


고객들의 연평균 잔고와 가입여부 분포를 시각화

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 'balance' 변수와 'y' 변수 추출
balance_data = Original['balance']
deposit_data = Original['y']

# 폰트 경로 설정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 데이터를 정기예금 가입 여부에 따라 그룹으로 나눔
groups = Original.groupby(deposit_data)

# 그룹화된 데이터에서 연평균잔고(balance)를 추출하여 리스트로 저장
data = [group[1]['balance'] for group in groups]

# 박스 플롯으로 그래프 그리기
plt.boxplot(data, labels=['No Deposit', 'Deposit'])

# 그래프 타이틀과 라벨링 추가
plt.title("Balance Distribution by Deposit", fontproperties=fontprop)
plt.xlabel("Deposit", fontproperties=fontprop)
plt.ylabel("Balance", fontproperties=fontprop)

# 그래프 출력
plt.show()



'최종학력' 변수를 시각화하여 표현

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 'education' 변수를 최종학력으로 변경하여 education_data에 저장
education_data = Original['education'].replace({
    'basic.4y': 'Primary Education',
    'basic.6y': 'Primary Education',
    'basic.9y': 'Basic Education',
    'high.school': 'High School',
    'university.degree': 'University Degree',
    'professional.course': 'Professional Course',
    'unknown': 'Unknown'
})

# 최종학력 별 데이터 개수 카운트
education_counts = education_data.value_counts()

# 폰트 경로 지정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# 그래프 생성
plt.bar(education_counts.index, education_counts.values)
plt.xlabel('Education Level', fontproperties=fontprop)
plt.ylabel('Count', fontproperties=fontprop)
plt.title('Distribution of Education Level', fontproperties=fontprop)
plt.xticks(rotation=45, fontproperties=fontprop) #45도로 회전
plt.show()


변수들을 시각화로 확인 후, 어떤 변수가 결과값인 'y'에 더 영향을 미치는지 확인하고자 함

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 수치형 변수만 선택
numeric_columns = Original.select_dtypes(include=['int64', 'float64']).columns

# 수치형 변수들 간의 상관관계 계산
correlation_matrix = Original[numeric_columns].corr()

# 히트맵 그리기
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()



변수들간의 상관관계를 보면 age와 balance의 상관관계는 0.08로 거의 없고 pday(캠페인의 마지막 연락 후 이번 캠페 인에서 다시 연락 받을 때까지 일수)과 previous(이번 캠페인동안 연락한 횟수) 변수간의 상관관계는 0.58로 상당히 높은것으로 보인다.

In [ ]:
import pandas as pd
import matplotlib.font_manager as fm

# 폰트 경로 지정
font_path = '/content/drive/MyDrive/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 정기예금 가입 여부에 따른 numeric feature들의 평균과 표준편차 계산
numeric_columns = Original.select_dtypes(include=['int64', 'float64']).columns
stats_by_deposit = Original.groupby('y')[numeric_columns].agg(['mean', 'std'])

# 표 형태로 결과 출력
stats_table = pd.DataFrame()
for col in stats_by_deposit.columns.levels[0]:
    stats_table[(col, 'Mean')] = stats_by_deposit[col, 'mean']
    stats_table[(col, 'Standard Deviation')] = stats_by_deposit[col, 'std']

stats_table.index = ['No Deposit', 'Deposit']
stats_table = stats_table.round(2)
print(stats_table)





In [ ]:
#numeric feature들의 상관관계 확인
import pandas as pd

# bank.csv 파일을 읽어와서 Original에 저장
Original = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 정기예금 가입 여부에 따른 numeric feature들의 평균과 표준편차 계산
numeric_columns = Original.select_dtypes(include=['int64', 'float64']).columns
stats_by_deposit = Original.groupby('y')[numeric_columns].agg(['mean', 'std'])

# 결과 출력
print(stats_by_deposit)


*평균(mean)은 변수의 값들의 평균적인 크기를 나타냄
정기예금 가입 결과에 따라 'no=정기예금 미가입', 'yes=정기예금 가입'로 나뉘어진다.

*표준편차(std)는 변수의 값들이 얼마나 퍼져있는지를 나타냄
 작을수록 변수 값들이 평균 주변에 모여있는 것을 의미하며, 클수록 변수 값들이 평균으로부터 더 멀리 퍼져있는 것을 의미함.

Duration 고객과의 평균 통화 시간은 "예" 그룹(552.742802)이 "아니오" 그룹(226.347500)보다 훨씬 길다. 이는 접촉 시간이 길수록 정기예금에 가입할 가능성이 높을 수 있음을 시사함

Previous(캠페인 이전에 연락받았던 횟수): 평균 연락 수는 "아니오" 그룹(0.471250)에 비해 "예" 그룹(1.090211)에서 더 높습니다. 이는 정기예금에 가입한 고객일수록 이전 컨택이 많았음을 의미하며, 이는 가입 결정에 영향을 미칠 수 있음을 시사함



<모델별 실습>
데이터세트 분리 : 학습데이터와 테스트 데이터로 분리
모델학습: 학습데이터를 기반으로 머신러닝 알고리즘을 적용하여 모델을 학습
예측수행: 학습된 모델을 이용하여 테스트 데이터의 결과 확인
평가: 예측된 결과값과 테스트 데이터의 실제 결과값을 비교해 모델 성능 평가




앙상블 모델

In [ ]:
#Data Splitting Libraries
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
#Classification Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier


데이터 전처리

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 데이터 불러오기
test_data = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 1. 결측치 처리
test_data = test_data.dropna()

# 2. 범주형 변수 인코딩
categorical_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
label_encoder = LabelEncoder()
for col in categorical_cols:
    test_data[col] = label_encoder.fit_transform(test_data[col])

# 3. 이상치 처리 (예시: 수치형 변수에서 3표준편차를 벗어나는 값 제거)
numeric_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
for col in numeric_cols:
    mean = test_data[col].mean()
    std = test_data[col].std()
    upper_bound = mean + 3 * std
    lower_bound = mean - 3 * std
    test_data = test_data[(test_data[col] >= lower_bound) & (test_data[col] <= upper_bound)]

# 4. 데이터 스케일링/정규화 (예시: Min-Max 스케일링)
numeric_cols = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
for col in numeric_cols:
    test_data[col] = (test_data[col] - test_data[col].min()) / (test_data[col].max() - test_data[col].min())

# 전처리된 데이터 확인
print("전처리된 데이터셋:")
print(test_data.head())


앙상블 모델링

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

# 전처리된 데이터 불러오기
test_data = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

# 독립 변수와 종속 변수 분리
X = test_data.drop('y', axis=1)
y = test_data['y']

# 문자열 값을 숫자로 변환
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # 'no'를 0으로, 'yes'를 1로 변환

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = label_encoder.fit_transform(X[col])

# 앙상블 모델 구축 (랜덤 포레스트, 그래디언트 부스팅, XGBoost)
random_forest = RandomForestClassifier()
gradient_boosting = GradientBoostingClassifier()
xgboost = XGBClassifier()

# 10-fold cross-validation 수행
rf_scores = cross_val_score(random_forest, X, y, cv=10)
gb_scores = cross_val_score(gradient_boosting, X, y, cv=10)
xgb_scores = cross_val_score(xgboost, X, y, cv=10)

# 각 모델의 평균 정확도 계산
rf_mean_accuracy = rf_scores.mean()
gb_mean_accuracy = gb_scores.mean()
xgb_mean_accuracy = xgb_scores.mean()

# 결과 출력
print("Random Forest 평균 정확도:", rf_mean_accuracy)
print("Gradient Boosting 평균 정확도:", gb_mean_accuracy)
print("XGBoost 평균 정확도:", xgb_mean_accuracy)




모델링 결과 시각화

ROC Curve로 표현

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve, auc

# Random Forest ROC 커브
y_pred_rf = cross_val_predict(random_forest, X, y, cv=10, method='predict_proba')[:, 1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y, y_pred_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

# Gradient Boosting ROC 커브
y_pred_gb = cross_val_predict(gradient_boosting, X, y, cv=10, method='predict_proba')[:, 1]
fpr_gb, tpr_gb, thresholds_gb = roc_curve(y, y_pred_gb)
roc_auc_gb = auc(fpr_gb, tpr_gb)

# XGBoost ROC 커브
y_pred_xgb = cross_val_predict(xgboost, X, y, cv=10, method='predict_proba')[:, 1]
fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(y, y_pred_xgb)
roc_auc_xgb = auc(fpr_xgb, tpr_xgb)

# ROC 커브 시각화
plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label='Random Forest (AUC = %0.2f)' % roc_auc_rf)
plt.plot(fpr_gb, tpr_gb, label='Gradient Boosting (AUC = %0.2f)' % roc_auc_gb)
plt.plot(fpr_xgb, tpr_xgb, label='XGBoost (AUC = %0.2f)' % roc_auc_xgb)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('거짓 양성 비율 (FPR)')
plt.ylabel('진짜 양성 비율 (TPR)')
plt.title('Receiver Operating Characteristic (ROC) 커브')
plt.legend(loc='lower right')
plt.show()


In [ ]:
#Random Forest의 AUC 값이 0.91이고 Gradient Boost와 XGBoost의 AUC 값이 0.90인 경우, 이는 모든 모델이 좋은 예측 성능을 가지고 있다는 것을 나타냄
#AUC 값이 0.9 이상이라면, 모델이 양성과 음성 샘플을 잘 구분하고 있다는 것을 의미한다.
#이 모델들은 높은 정확도로 정기예금 가입 여부를 예측할 수 있다고 해석할 수 있음
#또한, Random Forest의 AUC 값이 0.91로 약간 더 높으므로, Random Forest 모델이 다른 모델들보다 약간 더 우수한 성능을 가지고 있다고 할 수 있다.

In [ ]:
import matplotlib.pyplot as plt

# 각 모델의 평균 정확도 데이터
models = ['Random Forest', 'Gradient Boosting', 'XGBoost']
accuracies = [rf_mean_accuracy, gb_mean_accuracy, xgb_mean_accuracy]

# 시각화
plt.bar(models, accuracies)
plt.ylim(0.88, 0.91)  # y축 범위 설정 (수정 가능)
plt.xlabel('앙상블 모델')
plt.ylabel('평균 정확도')
plt.title('앙상블 모델의 평균 정확도')
plt.show()


In [ ]:
#어떤 변수가 영향을 많이 줬는지 확인
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

# 전처리된 데이터 불러오기
test_data = pd.read_csv('/content/drive/MyDrive/bank.csv', delimiter=';')

#Random Forest 모델의 feature_importances
random_forest.fit(X, y)
importances_rf = random_forest.feature_importances_
print("Random Forest 변수 중요도:")
for feature, importance in zip(X.columns, importances_rf):
    print(f"{feature}: {importance}")

#Gradient Boosting 모델의 feature_importances
gradient_boosting.fit(X, y)
importances_gb = gradient_boosting.feature_importances_
print("Gradient Boosting 변수 중요도:")
for feature, importance in zip(X.columns, importances_gb):
    print(f"{feature}: {importance}")

#XGBoost 모델의 feature_importances
xgboost.fit(X, y)
importances_xgb = xgboost.feature_importances_
print("XGBoost 변수 중요도:")
for feature, importance in zip(X.columns, importances_xgb):
    print(f"{feature}: {importance}")


In [ ]:
import matplotlib.pyplot as plt

# Random Forest 변수 중요도 시각화
plt.figure(figsize=(10, 6))
plt.bar(X.columns, importances_rf)
plt.xlabel('변수')
plt.ylabel('중요도')
plt.title('Random Forest 변수 중요도')
plt.xticks(rotation=90)
plt.show()

# Gradient Boosting 변수 중요도 시각화
plt.figure(figsize=(10, 6))
plt.bar(X.columns, importances_gb)
plt.xlabel('변수')
plt.ylabel('중요도')
plt.title('Gradient Boosting 변수 중요도')
plt.xticks(rotation=90)
plt.show()

# XGBoost 변수 중요도 시각화
plt.figure(figsize=(10, 6))
plt.bar(X.columns, importances_xgb)
plt.xlabel('변수')
plt.ylabel('중요도')
plt.title('XGBoost 변수 중요도')
plt.xticks(rotation=90)
plt.show()

# 모델별 정확도 결과 시각화
models = ['Random Forest', 'Gradient Boosting', 'XGBoost']
accuracies = [rf_mean_accuracy, gb_mean_accuracy, xgb_mean_accuracy]

plt.figure(figsize=(8, 5))
plt.bar(models, accuracies)
plt.xlabel('모델')
plt.ylabel('정확도')
plt.title('각 모델의 정확도')
plt.ylim([0.85, 0.9])
plt.show()


Confusion matrix로 표현

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
import seaborn as sns

# 예측값 계산
y_pred_rf = cross_val_predict(random_forest, X, y, cv=10)
y_pred_gb = cross_val_predict(gradient_boosting, X, y, cv=10)
y_pred_xgb = cross_val_predict(xgboost, X, y, cv=10)

# Confusion matrix 계산
cm_rf = confusion_matrix(y, y_pred_rf)
cm_gb = confusion_matrix(y, y_pred_gb)
cm_xgb = confusion_matrix(y, y_pred_xgb)

# Confusion matrix 시각화
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues')
plt.title('Random Forest')
plt.xlabel('Predicted')
plt.ylabel('True')

plt.subplot(1, 3, 2)
sns.heatmap(cm_gb, annot=True, fmt='d', cmap='Blues')
plt.title('Gradient Boosting')
plt.xlabel('Predicted')
plt.ylabel('True')

plt.subplot(1, 3, 3)
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Blues')
plt.title('XGBoost')
plt.xlabel('Predicted')
plt.ylabel('True')

plt.tight_layout()
plt.show()
#True Positive (TP): 실제 Positive인 샘플을 Positive로 정확히 예측한 경우
#False Positive (FP): 실제 Negative인 샘플을 Positive로 잘못 예측한 경우
#False Negative (FN): 실제 Positive인 샘플을 Negative로 잘못 예측한 경우
#True Negative (TN): 실제 Negative인 샘플을 Negative로 정확히 예측한 경우